In [1]:
import json
import os
import pandas as pd

def get_box_pos(bbox,orig_size=[6000,4000]):
    """
    inputs:
        bbox 框的位置 xywh
        orig_size 图像大小 x,y heng shu
    outputs:
        框所在图片位置
        左上角，上部，右上角
        左部，  中部，右部
        左下角  下部  右下角
        012
        345
        678
           
    """
    #center=((bbox[0]+bbox[2])/2,(bbox[1]+bbox[3])/2)    ##获取中心坐标
    center=(bbox[0],bbox[1])

    if center[0]<orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top left corner'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top'
    if center[0]>2*orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top right corner'
    if center[0]<orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'left'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'center'
    if center[0]>2*orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'right'
    if center[0]<orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower left corner'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower'
    if center[0]>2*orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower right corner'

In [2]:
def deal_json(load_dir='annotations.json'):

    data=json.load(open(load_dir))

    bbox_info=data['annotations']

    bbox_list = [[[],[]] for _ in range(400)]            ## 每张图的bbox

    for bbox in bbox_info:
        #if bbox['image_id']==278 or bbox['image_id']==371:
        #    continue
        bbox_list[bbox['image_id']][0].append(get_box_pos(bbox['bbox']))
        bbox_list[bbox['image_id']][1].append(data['categories'][bbox['category_id']]['name'])

    bbox_list.pop(371)
    bbox_list.pop(278)           ##这两个位置的图片没有对应的rel标签

    return bbox_list

In [3]:
bbox_list=deal_json()

In [6]:
import re
def remove_chinese_and_punctuation(text):
    # 定义一个正则表达式模式，匹配中文字符和括号
    c_list=[]
    pattern =  r'[\u4e00-\u9fa5（）.,;:!?—–—《》“”‘’「」『』【】「」〈〉<>………，；()。、]'
    for i in text:
    
    # 使用 re.sub 替换匹配到的字符为空字符
        i=i.replace('\xa0', ' ')
        cleaned_text = re.sub(pattern, '', i)

        c_list.append(cleaned_text.rstrip())
    
    return c_list

def get_ground_truth(bbox_list,rel_dir):
    """生成ground_truth"""
    ground_list=[]

    dir_list=os.listdir(rel_dir)



    for k,single_bbox in enumerate(bbox_list):        ##便利每一张图的box
        ground={
            "number":{},
            "location":{},
            "relationship":{}
        }

        count_dict={}                          ###数量
        for item in single_bbox[1]:
            if item in count_dict:
                count_dict[item] += 1
            else:
                count_dict[item] = 1

        keys=count_dict.keys()                   ### 位置
        loc_dict={key: [] for key in keys}

        for i in range(len(single_bbox[0])):
            loc_dict[single_bbox[1][i]].append(single_bbox[0][i])

        print(dir_list[k])
        df=pd.read_excel(os.path.join(rel_dir,dir_list[k]),header=None,engine="openpyxl")


        rel_list=[]
        for j in range(len(df)):

            rel_list.append([df.loc[j][0],remove_chinese_and_punctuation([df.loc[j][2]])[0],df.loc[j][3]])    

        ground['number']=count_dict
        ground['location']=loc_dict
        ground['relationship']=rel_list

        ground_list.append(ground)

    return ground_list
    
        



In [7]:
ground_list=get_ground_truth(bbox_list,'r_total')

000.xlsx
001.xlsx
002.xlsx
003.xlsx
004.xlsx
005.xlsx
006.xlsx
008.xlsx
011.xlsx
013.xlsx
014.xlsx
015.xlsx
016.xlsx
018.xlsx
019.xlsx
021.xlsx
022.xlsx
023.xlsx
026.xlsx
028.xlsx
031.xlsx
035.xlsx
038.xlsx
040.xlsx
041.xlsx
042.xlsx
043.xlsx
044.xlsx
045.xlsx
047.xlsx
049.xlsx
051.xlsx
052.xlsx
053.xlsx
055.xlsx
056.xlsx
057.xlsx
058.xlsx
059.xlsx
060.xlsx
062.xlsx
063.xlsx
065.xlsx
068.xlsx
070.xlsx
071.xlsx
073.xlsx
074.xlsx
075.xlsx
077.xlsx
078.xlsx
079.xlsx
080.xlsx
081.xlsx
083.xlsx
086.xlsx
088.xlsx
089.xlsx
092.xlsx
095.xlsx
098.xlsx
099.xlsx
100.xlsx
101.xlsx
102.xlsx
103.xlsx
104.xlsx
106.xlsx
107.xlsx
109.xlsx
110.xlsx
111.xlsx
112.xlsx
113.xlsx
116.xlsx
117.xlsx
118.xlsx
119.xlsx
120.xlsx
121.xlsx
122.xlsx
123.xlsx
124.xlsx
126.xlsx
128.xlsx
130.xlsx
133.xlsx
134.xlsx
135.xlsx
136.xlsx
137.xlsx
138.xlsx
139.xlsx
140.xlsx
141.xlsx
145.xlsx
146.xlsx
147.xlsx
148.xlsx
149.xlsx
150.xlsx
153.xlsx
154.xlsx
155.xlsx
156.xlsx
157.xlsx
158.xlsx
159.xlsx
160.xlsx
161.xlsx
162.xlsx
1

In [8]:
import pickle 

with open('ground_truth.pickle','wb') as f:
    pickle.dump(ground_list,f)

In [18]:
import os

img_list=os.listdir('JPEGImages')
xlsx_list=os.listdir('r_total')

In [19]:
img_=[]
xlsx_=[]

for i in img_list:
    img_.append(i[:3])

for  i in xlsx_list:
    xlsx_.append(i[:3])

In [21]:
for i in img_:
    if i not in xlsx_:
        print(i)

420
561


In [15]:
get_ground_truth

<function __main__.get_ground_truth(bbox_list, rel_dir)>